<a href="https://colab.research.google.com/github/sergey151004/1/blob/main/pvs_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!apt install nvidia-cuda-toolkit

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
nvidia-cuda-toolkit is already the newest version (11.5.1-1ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.


In [14]:
%%writefile integral.cu
#include <stdio.h>
#include <math.h>
#include <chrono>

#define PI 3.14159265358979323846

// Оригинальная сложная функция для интегрирования
__device__ double ComputeOriginalFunction(double x) {
  if (x < 1e-10 || x > PI - 1e-10) return 0.0;
  double sin_x = sin(x);
  if (fabs(sin_x) < 1e-12) return 0.0;
  double cot_x = cos(x) / sin_x;
  double sin_term = 1.0 + sin_x;
  if (sin_term < 1e-12) return 0.0;
  double ln_term = log(sin_term);
  double sin_ln_term = sin(sin_term);
  double denominator = ln_term * sin_ln_term;
  if (fabs(denominator) < 1e-100) return 0.0;
  return cot_x / denominator;
}

__global__ void ComputeAndReduce(double a, double dx, int n, double* result) {
  int idx = blockIdx.x * blockDim.x + threadIdx.x;
  double sum = 0.0;
  if (idx < n) {
    double x = a + (idx + 0.5) * dx;
    sum = ComputeOriginalFunction(x);
  }

  extern __shared__ double sdata[];
  int tid = threadIdx.x;
  sdata[tid] = sum;
  __syncthreads();

  for (int s = blockDim.x / 2; s > 0; s >>= 1) {
    if (tid < s) {
      sdata[tid] += sdata[tid + s];
    }
    __syncthreads();
  }

  if (tid == 0) {
    result[blockIdx.x] = sdata[0];
  }
}

__global__ void ReduceSum(double* input, double* output, int n) {
  extern __shared__ double sdata[];
  int tid = threadIdx.x;
  int i = blockIdx.x * blockDim.x + tid;
  sdata[tid] = (i < n) ? input[i] : 0.0;
  __syncthreads();

  for (int s = blockDim.x / 2; s > 0; s >>= 1) {
    if (tid < s) {
      sdata[tid] += sdata[tid + s];
    }
    __syncthreads();
  }

  if (tid == 0) {
    output[blockIdx.x] = sdata[0];
  }
}

double ComputeIntegral(double a, double b, int n, double* compute_time) {
  cudaEvent_t start, stop;
  cudaEventCreate(&start);
  cudaEventCreate(&stop);

  cudaEventRecord(start);

  double dx = (b - a) / n;
  int threads = 256;
  int blocks = (n + threads - 1) / threads;

  double* d_partial_sums;
  cudaMalloc(&d_partial_sums, sizeof(double) * blocks);

  ComputeAndReduce<<<blocks, threads, threads * sizeof(double)>>>(a, dx, n, d_partial_sums);

  cudaError_t err = cudaGetLastError();
  if (err != cudaSuccess) {
    printf("CUDA error: %s\n", cudaGetErrorString(err));
    return NAN;
  }

  int n_blocks = blocks;
  double* d_sum = d_partial_sums;
  double* d_temp;
  while (n_blocks > 1) {
    int n_threads = threads;
    int n_blocks_new = (n_blocks + n_threads - 1) / n_threads;
    cudaMalloc(&d_temp, sizeof(double) * n_blocks_new);
    ReduceSum<<<n_blocks_new, n_threads, n_threads * sizeof(double)>>>(d_sum, d_temp, n_blocks);
    cudaFree(d_sum);
    d_sum = d_temp;
    n_blocks = n_blocks_new;
  }

  double total_sum;
  cudaMemcpy(&total_sum, d_sum, sizeof(double), cudaMemcpyDeviceToHost);
  double integral = total_sum * dx;

  cudaEventRecord(stop);
  cudaEventSynchronize(stop);
  float milliseconds = 0;
  cudaEventElapsedTime(&milliseconds, start, stop);
  *compute_time = milliseconds / 1000.0; // Convert to seconds

  cudaFree(d_sum);
  cudaEventDestroy(start);
  cudaEventDestroy(stop);

  return integral;
}

int main() {
  double a = 0.1;
  double b = PI - 0.1;
  int n = 100000000;

  double compute_time = 0.0;
  double integral = ComputeIntegral(a, b, n, &compute_time);

  printf("============ INTEGRATION RESULTS ============\n");
  printf("Integral value:    %.12f\n", integral);
  printf("Absolute error:    %.3e\n", fabs(integral));
  printf("Computation time:  %.3f seconds\n", compute_time);
  printf("\n");
  printf("Parameters:\n");
  printf("  Interval:       [%.3f, %.3f]\n", a, b);
  printf("  Length:         %.5f\n", b - a);
  printf("  Segments:       %d\n", n);
  printf("  Points:         %.0f million\n", n / 1e6);
  printf("  Symmetric mode: %s\n", "false");
  printf("============================================\n");

  double points_per_second = n / compute_time;
  printf("\nPerformance metrics:\n");
  printf("  Throughput:      %.1f million points/second\n", points_per_second / 1e6);
  printf("  Time per point:  %.3f nanoseconds\n", (compute_time * 1e9) / n);

  return 0;
}


Overwriting integral.cu


In [15]:
!nvcc -arch=sm_75 -o integral integral.cu

In [16]:
!./integral

============ INTEGRATION RESULTS ============
Integral value:    0.000000000000
Absolute error:    2.751e-14
Computation time:  0.375 seconds

Parameters:
  Interval:       [0.100, 3.042]
  Length:         2.94159
  Segments:       100000000
  Points:         100 million
  Symmetric mode: true

Performance metrics:
  Throughput:      266.6 million points/second
  Time per point:  3.751 nanoseconds
